In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, log_loss
import operator
import json
from IPython import display
import os
import warnings

np.random.seed(0)
warnings.filterwarnings("ignore")
THRESHOLD = 4

Task: To predict whether the user likes the mobile phone or not. <br>
Assumption: If the average rating of mobile >= threshold, then the user likes it, otherwise not.

<b>Missing values:</b><br>
'Also Known As'(459),'Applications'(421),'Audio Features'(437),'Bezel-less display'(266),'Browser'(449),'Build Material'(338),'Co-Processor'(451),'Display Colour'(457),'Mobile High-Definition Link(MHL)'(472),'Music'(447)
'Email','Fingerprint Sensor Position'(174),'Games'(446),'HDMI'(454),'Heart Rate Monitor'(467),'IRIS Scanner'(467),
'Optical Image Stabilisation'(219),'Other Facilities'(444),'Phone Book'(444),'Physical Aperture'(87),'Quick Charging'(122),'Ring Tone'(444),'Ruggedness'(430),SAR Value(315),'SIM 3'(472),'SMS'(470)', 'Screen Protection'(229),'Screen to Body Ratio (claimed by the brand)'(428),'Sensor'(242),'Software Based Aperture'(473),
'Special Features'(459),'Standby time'(334),'Stylus'(473),'TalkTime'(259), 'USB Type-C'(374),'Video Player'(456),
'Video Recording Features'(458),'Waterproof'(398),'Wireless Charging','USB OTG Support'(159), 'Video ,'Recording'(113),'Java'(471),'Browser'(448)

<b>Very low variance:</b><br>
'Architecture'(most entries are 64-bit),'Audio Jack','GPS','Loudspeaker','Network','Network Support','Other Sensors'(28),'SIM Size', 'VoLTE'


<b>Multivalued:</b><br>
'Colours','Custom UI','Model'(1),'Other Sensors','Launch Date'

<b>Not important:</b><br>
'Bluetooth', 'Settings'(75),'Wi-Fi','Wi-Fi Features'

<b>Doubtful:</b><br>
'Aspect Ratio','Autofocus','Brand','Camera Features','Fingerprint Sensor'(very few entries are missing),
'Fingerprint Sensor Position', 'Graphics'(multivalued),'Image resolution'(multivalued),'SIM Size','Sim Slot(s)', 'User Available Storage', 'SIM 1', 'SIM 2','Shooting Modes', 'Touch Screen'(24), 'USB Connectivity'
    
<b>To check:</b><br>
'Display Type','Expandable Memory','FM Radio'

<b>High Correlation with other features</b><br>
'SIM Slot(s)' high correlation with SIM1
'Weight' has high high correlation with capacity , screen-to-body ratio
'Height' - screen size is also there
    
<b>Given a mobile, we can't directly get these features</b><br>
'Rating Count', 'Review Count'

<b>Keeping:</b><br>
'Capacity','Flash'(17),'Height'(22),'Internal Memory'(20, require cleaning),'Operating System'(25, require cleaning), 'Pixel Density'(1, clean it),'Processor'(22, clean it), 'RAM'(17, clean), 'Rating','Resolution'(cleaning), 'Screen Resolution','Screen Size', 'Thickness'(22), 'Type','User Replaceable','Weight'(cleaning),'Sim Size'(), 'Other Sensors'(28), 'Screen to Body Ratio (calculated)','Width',


In [2]:
# read data from file
train = pd.read_csv("../input/train.csv") 
test = pd.read_csv("../input/test.csv")

# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 355
Number of features in train: 99
Number of data points in test: 119
Number of features in test: 98


In [3]:
def data_clean(data):
    
    # Let's first remove all missing value features
    columns_to_remove = ['Also Known As','Applications','Audio Features','Bezel-less display'
                         'Browser','Build Material','Co-Processor','Browser'
                         'Display Colour','Mobile High-Definition Link(MHL)',
                         'Music', 'Email','Fingerprint Sensor Position',
                         'Games','HDMI','Heart Rate Monitor','IRIS Scanner', 
                         'Optical Image Stabilisation','Other Facilities',
                         'Phone Book','Physical Aperture','Quick Charging',
                         'Ring Tone','Ruggedness','SAR Value','SIM 3','SMS',
                         'Screen Protection','Screen to Body Ratio (claimed by the brand)',
                         'Sensor','Software Based Aperture', 'Special Features',
                         'Standby time','Stylus','TalkTime', 'USB Type-C',
                         'Video Player', 'Video Recording Features','Waterproof',
                         'Wireless Charging','USB OTG Support', 'Video Recording','Java']

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    #Features having very low variance 
    columns_to_remove = ['Architecture','Audio Jack','GPS','Loudspeaker','Network','Network Support','VoLTE']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    # Multivalued:
    columns_to_remove = ['Architecture','Launch Date','Audio Jack','GPS','Loudspeaker','Network','Network Support','VoLTE', 'Custom UI']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    # Not much important
    columns_to_remove = ['Bluetooth', 'Settings','Wi-Fi','Wi-Fi Features']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]
    
    return data

# Removing features

In [4]:
train = data_clean(train)
test = data_clean(test)

removing all those data points in which more than 15 features are missing 

In [5]:
train = train[(train.isnull().sum(axis=1) <= 15)]
# You shouldn't remove data points from test set
#test = test[(test.isnull().sum(axis=1) <= 15)]

In [6]:
# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 341
Number of features in train: 47
Number of data points in test: 119
Number of features in test: 46


# Filling Missing values

In [7]:
def for_integer(test):
    try:
        test = test.strip()
        return int(test.split(' ')[0])
    except IOError:
           pass
    except ValueError:
        pass
    except:
        pass

def for_string(test):
    try:
        test = test.strip()
        return (test.split(' ')[0])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass

def for_float(test):
    try:
        test = test.strip()
        return float(test.split(' ')[0])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass
def find_freq(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[2][0] == '(':
            return float(test[2][1:])
        return float(test[2])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass

    
def for_Internal_Memory(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[1] == 'GB':
            return int(test[0])
        if test[1] == 'MB':
#             print("here")
            return (int(test[0]) * 0.001)
    except IOError:
           pass
    except ValueError:
        pass
    except:
        pass
    
def find_freq(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[2][0] == '(':
            return float(test[2][1:])
        return float(test[2])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass


In [8]:
def data_clean_2(x):
    data = x.copy()
    
    data['Capacity'] = data['Capacity'].apply(for_integer)

    data['Height'] = data['Height'].apply(for_float)
    data['Height'] = data['Height'].fillna(data['Height'].mean())

    data['Internal Memory'] = data['Internal Memory'].apply(for_Internal_Memory)

    data['Pixel Density'] = data['Pixel Density'].apply(for_integer)

    data['Internal Memory'] = data['Internal Memory'].fillna(data['Internal Memory'].median())
    data['Internal Memory'] = data['Internal Memory'].astype(int)

    data['RAM'] = data['RAM'].apply(for_integer)
    data['RAM'] = data['RAM'].fillna(data['RAM'].median())
    data['RAM'] = data['RAM'].astype(int)

    data['Resolution'] = data['Resolution'].apply(for_integer)
    data['Resolution'] = data['Resolution'].fillna(data['Resolution'].median())
    data['Resolution'] = data['Resolution'].astype(int)

    data['Screen Size'] = data['Screen Size'].apply(for_float)

    data['Thickness'] = data['Thickness'].apply(for_float)
    data['Thickness'] = data['Thickness'].fillna(data['Thickness'].mean())
    data['Thickness'] = data['Thickness'].round(2)

    data['Type'] = data['Type'].fillna('Li-Polymer')

    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].apply(for_float)
    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].fillna(data['Screen to Body Ratio (calculated)'].mean())
    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].round(2)

    data['Width'] = data['Width'].apply(for_float)
    data['Width'] = data['Width'].fillna(data['Width'].mean())
    data['Width'] = data['Width'].round(2)

    data['Flash'][data['Flash'].isna() == True] = "Other"

    data['User Replaceable'][data['User Replaceable'].isna() == True] = "Other"

    data['Num_cores'] = data['Processor'].apply(for_string)
    data['Num_cores'][data['Num_cores'].isna() == True] = "Other"


    data['Processor_frequency'] = data['Processor'].apply(find_freq)
    #because there is one entry with 208MHz values, to convert it to GHz
    data['Processor_frequency'][data['Processor_frequency'] > 200] = 0.208
    data['Processor_frequency'] = data['Processor_frequency'].fillna(data['Processor_frequency'].mean())
    data['Processor_frequency'] = data['Processor_frequency'].round(2)

    data['Camera Features'][data['Camera Features'].isna() == True] = "Other"

    #simplifyig Operating System to os_name for simplicity
    data['os_name'] = data['Operating System'].apply(for_string)
    data['os_name'][data['os_name'].isna() == True] = "Other"

    data['Sim1'] = data['SIM 1'].apply(for_string)

    data['SIM Size'][data['SIM Size'].isna() == True] = "Other"

    data['Image Resolution'][data['Image Resolution'].isna() == True] = "Other"

    data['Fingerprint Sensor'][data['Fingerprint Sensor'].isna() == True] = "Other"

    data['Expandable Memory'][data['Expandable Memory'].isna() == True] = "No"

    data['Weight'] = data['Weight'].apply(for_integer)
    data['Weight'] = data['Weight'].fillna(data['Weight'].mean())
    data['Weight'] = data['Weight'].astype(int)

    data['SIM 2'] = data['SIM 2'].apply(for_string)
    data['SIM 2'][data['SIM 2'].isna() == True] = "Other"
    
    return data

In [9]:
train = data_clean_2(train)
test = data_clean_2(test)

# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 341
Number of features in train: 51
Number of data points in test: 119
Number of features in test: 50


Not very important feature

In [10]:
def data_clean_3(x):
    
    data = x.copy()

    columns_to_remove = ['User Available Storage','SIM Size','Chipset','Processor','Autofocus','Aspect Ratio','Touch Screen',
                        'Bezel-less display','Operating System','SIM 1','USB Connectivity','Other Sensors','Graphics','FM Radio',
                        'NFC','Shooting Modes','Browser','Display Colour' ]

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]


    columns_to_remove = [ 'Screen Resolution','User Replaceable','Camera Features',
                        'Thickness', 'Display Type']

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]


    columns_to_remove = ['Fingerprint Sensor', 'Flash', 'Rating Count', 'Review Count','Image Resolution','Type','Expandable Memory',\
                        'Colours','Width','Model']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    return data

In [11]:
train = data_clean_3(train)
test = data_clean_3(test)


# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])


Number of data points in train: 341
Number of features in train: 18
Number of data points in test: 119
Number of features in test: 17


In [12]:
# one hot encoding

train_ids = train['PhoneId']
test_ids = test['PhoneId']

cols = list(test.columns)
cols.remove('PhoneId')
cols.insert(0, 'PhoneId')

combined = pd.concat([train.drop('Rating', axis=1)[cols], test[cols]])
print(combined.shape)
print(combined.columns)

combined = pd.get_dummies(combined)
print(combined.shape)
print(combined.columns)

train_new = combined[combined['PhoneId'].isin(train_ids)]
test_new = combined[combined['PhoneId'].isin(test_ids)]

(460, 17)
Index(['PhoneId', 'Screen Size', 'os_name', 'Num_cores', 'Height', 'RAM',
       'Brand', 'Processor_frequency', 'Sim1', 'SIM Slot(s)', 'Pixel Density',
       'SIM 2', 'Weight', 'Screen to Body Ratio (calculated)',
       'Internal Memory', 'Capacity', 'Resolution'],
      dtype='object')
(460, 87)
Index(['PhoneId', 'Screen Size', 'Height', 'RAM', 'Processor_frequency',
       'Pixel Density', 'Weight', 'Screen to Body Ratio (calculated)',
       'Internal Memory', 'Capacity', 'Resolution', 'os_name_Android',
       'os_name_Blackberry', 'os_name_KAI', 'os_name_Nokia', 'os_name_Other',
       'os_name_Tizen', 'os_name_iOS', 'Num_cores_312', 'Num_cores_Deca',
       'Num_cores_Dual', 'Num_cores_Hexa', 'Num_cores_Octa', 'Num_cores_Other',
       'Num_cores_Quad', 'Num_cores_Tru-Octa', 'Brand_10.or', 'Brand_Apple',
       'Brand_Asus', 'Brand_Billion', 'Brand_Blackberry', 'Brand_Comio',
       'Brand_Coolpad', 'Brand_Do', 'Brand_Gionee', 'Brand_Google',
       'Brand_HTC', 'Bra

In [13]:
train_new = train_new.merge(train[['PhoneId', 'Rating']], on='PhoneId')

In [14]:
# check the number of features and data points in train
print("Number of data points in train: %d" % train_new.shape[0])
print("Number of features in train: %d" % train_new.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test_new.shape[0])
print("Number of features in test: %d" % test_new.shape[1])

Number of data points in train: 341
Number of features in train: 88
Number of data points in test: 119
Number of features in test: 87


In [15]:
#train_new = train_new.drop(['Rating_x'], axis = 1)
#train_new.head()

# Save raw Train Dataframe for analysis and Visualisation
train_new.to_csv('../input/df_train_raw.csv', index=False)

# Save raw Test Dataframe for analysis and Visualisation
test_new.to_csv('../input/df_test_raw.csv', index=False)


In [16]:
test_new.head()

,PhoneId,Screen Size,Height,RAM,Processor_frequency,Pixel Density,Weight,Screen to Body Ratio (calculated),Internal Memory,Capacity,...,Sim1_3G,Sim1_4G,"SIM Slot(s)_Dual SIM, GSM+CDMA","SIM Slot(s)_Dual SIM, GSM+GSM","SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE","SIM Slot(s)_Single SIM, GSM",SIM 2_2G,SIM 2_3G,SIM 2_4G,SIM 2_Other
0,3,6.20,156.2,2,1.8,271,168,80.85,16,4230,...,0,1,0,0,1,0,0,0,1,0
1,11,6.20,156.0,4,1.8,402,205,81.60,64,5000,...,0,1,0,0,1,0,0,0,1,0
2,13,6.30,156.7,6,2.0,409,169,83.84,64,3500,...,0,1,0,0,1,0,0,0,1,0
3,16,6.30,156.7,4,2.0,409,169,83.84,64,3500,...,0,1,0,0,1,0,0,0,1,0
4,19,5.99,158.6,4,1.8,403,181,77.43,64,4000,...,0,1,0,0,1,0,0,0,1,0


## Normalising the Columns in Train DataFrame

In [17]:
train_new['Capacity'] = train_new['Capacity'].apply(lambda x: ( (x - train_new['Capacity'].min()) ) / ( (train_new['Capacity'].max()-train_new['Capacity'].min())))
train_new['Internal Memory'] = train_new['Internal Memory'].apply(lambda x: ( (x - train_new['Internal Memory'].min()) ) / ( (train_new['Internal Memory'].max()-train_new['Internal Memory'].min())))
train_new['RAM'] = train_new['RAM'].apply(lambda x: ( (x - train_new['RAM'].min()) ) / ( (train_new['RAM'].max()-train_new['RAM'].min())))
train_new['Pixel Density'] = train_new['Pixel Density'].apply(lambda x: ( (x - train_new['Pixel Density'].min()) ) / ( (train_new['Pixel Density'].max()-train_new['Pixel Density'].min())))
train_new['Weight'] = train_new['Weight'].apply(lambda x: ( (x - train_new['Weight'].min()) ) / ( (train_new['Weight'].max()-train_new['Weight'].min())))
#train_new['Processor_frequency'] = train_new['Processor_frequency'].apply(lambda x: ( (x - train_new['Processor_frequency'].min()) ) / ( (train_new['Processor_frequency'].max()-train_new['Processor_frequency'].min())))
train_new['Resolution'] = train_new['Resolution'].apply(lambda x: ( (x - train_new['Resolution'].min()) ) / ( (train_new['Resolution'].max()-train_new['Resolution'].min())))

#train_new['Screen Size'] = train_new['Screen Size'].apply(lambda x: ( (x - train_new['Screen Size'].min()) ) / ( (train_new['Screen Size'].max()-train_new['Screen Size'].min())))
train_new['Screen to Body Ratio (calculated)'] = train_new['Screen to Body Ratio (calculated)'].apply(lambda x: ( (x - train_new['Screen to Body Ratio (calculated)'].min()) ) / ( (train_new['Screen to Body Ratio (calculated)'].max()-train_new['Screen to Body Ratio (calculated)'].min())))


# Binarise the Rating to get approprite Label
train_new['Rating'] = train_new['Rating'].apply(lambda x: int(x>=4) )
#train_new = train_new.drop(['Rating_x'], axis = 1)
#train_new = train_new.drop(['Rating_x'], axis = 1)

#train_new = train_new.drop(['Rating_y'], axis = 1)

train_new.head()

,PhoneId,Screen Size,Height,RAM,Processor_frequency,Pixel Density,Weight,Screen to Body Ratio (calculated),Internal Memory,Capacity,...,Sim1_4G,"SIM Slot(s)_Dual SIM, GSM+CDMA","SIM Slot(s)_Dual SIM, GSM+GSM","SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE","SIM Slot(s)_Single SIM, GSM",SIM 2_2G,SIM 2_3G,SIM 2_4G,SIM 2_Other,Rating
0,0,6.26,157.9,0.005871,1.8,0.600462,0.455882,0.891040,0.1250,0.248748,...,1,0,0,1,0,0,0,1,0,1
1,1,6.20,156.2,0.003914,1.8,0.295612,0.404412,0.893693,0.0625,0.267947,...,1,0,0,1,0,0,0,1,0,1
2,2,6.30,157.0,0.003914,2.1,0.614319,0.404412,0.937871,0.0625,0.207012,...,1,0,0,1,0,0,0,1,0,1
3,4,6.00,159.8,0.005871,2.2,0.618938,0.408088,0.798938,0.1250,0.190317,...,1,0,1,0,0,0,0,1,0,1
4,5,6.50,160.4,0.005871,2.2,0.584296,0.430147,0.946456,0.1250,0.227880,...,1,0,0,1,0,0,0,1,0,1


## Normalising the Columns in Test DataFrame

In [18]:
test_new['Capacity'] = test_new['Capacity'].apply(lambda x: ( (x - test_new['Capacity'].min()) ) / ( (test_new['Capacity'].max()-test_new['Capacity'].min())))
test_new['Internal Memory'] = test_new['Internal Memory'].apply(lambda x: ( (x - test_new['Internal Memory'].min()) ) / ( (test_new['Internal Memory'].max()-test_new['Internal Memory'].min())))
test_new['RAM'] = test_new['RAM'].apply(lambda x: ( (x - test_new['RAM'].min()) ) / ( (test_new['RAM'].max()-test_new['RAM'].min())))
test_new['Pixel Density'] = test_new['Pixel Density'].apply(lambda x: ( (x - test_new['Pixel Density'].min()) ) / ( (test_new['Pixel Density'].max()-test_new['Pixel Density'].min())))
test_new['Weight'] = test_new['Weight'].apply(lambda x: ( (x - test_new['Weight'].min()) ) / ( (test_new['Weight'].max()-test_new['Weight'].min())))
#test_new['Processor_frequency'] = test_new['Processor_frequency'].apply(lambda x: ( (x - test_new['Processor_frequency'].min()) ) / ( (test_new['Processor_frequency'].max()-test_new['Processor_frequency'].min())))
test_new['Resolution'] = test_new['Resolution'].apply(lambda x: ( (x - test_new['Resolution'].min()) ) / ( (test_new['Resolution'].max()-test_new['Resolution'].min())))

#test_new['Screen Size'] = test_new['Screen Size'].apply(lambda x: ( (x - test_new['Screen Size'].min()) ) / ( (test_new['Screen Size'].max()-test_new['Screen Size'].min())))
test_new['Screen to Body Ratio (calculated)'] = test_new['Screen to Body Ratio (calculated)'].apply(lambda x: ( (x - test_new['Screen to Body Ratio (calculated)'].min()) ) / ( (test_new['Screen to Body Ratio (calculated)'].max()-test_new['Screen to Body Ratio (calculated)'].min())))
test_new.drop(['Brand_Xiaomi Poco'], axis=1)
#test_new['']


test_new.head()

,PhoneId,Screen Size,Height,RAM,Processor_frequency,Pixel Density,Weight,Screen to Body Ratio (calculated),Internal Memory,Capacity,...,Sim1_3G,Sim1_4G,"SIM Slot(s)_Dual SIM, GSM+CDMA","SIM Slot(s)_Dual SIM, GSM+GSM","SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE","SIM Slot(s)_Single SIM, GSM",SIM 2_2G,SIM 2_3G,SIM 2_4G,SIM 2_Other
0,3,6.20,156.2,0.001957,1.8,0.248811,0.557522,0.931169,0.0625,0.806533,...,0,1,0,0,1,0,0,0,1,0
1,11,6.20,156.0,0.005871,1.8,0.456418,0.884956,0.942441,0.2500,1.000000,...,0,1,0,0,1,0,0,0,1,0
2,13,6.30,156.7,0.009785,2.0,0.467512,0.566372,0.976105,0.2500,0.623116,...,0,1,0,0,1,0,0,0,1,0
3,16,6.30,156.7,0.005871,2.0,0.467512,0.566372,0.976105,0.2500,0.623116,...,0,1,0,0,1,0,0,0,1,0
4,19,5.99,158.6,0.005871,1.8,0.458003,0.672566,0.879772,0.2500,0.748744,...,0,1,0,0,1,0,0,0,1,0


## End of Preprocessing

In [19]:
#submission.to_csv("submission.csv", index=False)

## Save the processed dataset into csv files

In [20]:
filePath_w_train = '../input/df_train_processed.csv'
train_new.to_csv(filePath_w_train, index = False)

In [21]:
filePath_w_test = '../input/df_test_processed.csv'
test_new.to_csv(filePath_w_test, index = False)

## Creating Vectors

In [22]:
# Importing Dependancies
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

In [23]:
# Excluding the Phone ID in both sets

X_train = train_new.iloc[:,1:].values
#print(X_train[0])

X_test = test_new.iloc[:,1:].values
#print(X_test[0])

# Saperating Features and Labels
Y_train = X_train[:,-1]
X_train = X_train[:,:-1]

In [24]:
# Printing diamentions of data
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
#print(Y_test.shape)

(341, 86)
(341,)
(119, 86)


# Building Model

In [25]:
###############################################################################

# Create Model Class
class Model(object):
    
    # constructor with weights
    def __init__(self):
        self.Weights = np.random.rand(86)
        self.b = 0

    # summation function / dot product
    def Activation(self, x):
        #print(x.shape)
        #print(self.Weights.shape)
        WX = np.dot(self.Weights, x)
        #print(WX)
        if WX > self.b :
            return 1
        else :
            return 0
        
        
    def Predict(self, X_data):
        # Arrary to store Output
        Y_predicted = []
        for x in X_data:
            r_xw = self.Activation(x)
            Y_predicted.append(r_xw)
            
        return np.array(Y_predicted)



    def Train(self, X_train, Y_train, epoches=10000, Learning_rate = 0.0001):

        Weight_M = []

        # Iterating upto epoches
        for e in range(epoches):
            # iterating through each rows
            for x_train, y_true in zip(X_train, Y_train):
                y_predicted = self.Activation(x_train)
        
                # updating Weights
                if y_predicted == 0 and y_true == 1 :
                    self.Weights = self.Weights + x_train*Learning_rate
                    
                elif y_predicted == 1 and y_true == 0 :
                    self.Weights = self.Weights - x_train*Learning_rate
                    
            # Save Weights for future predictions
            Weight_M.append(self.Weights)
            acc = accuracy_score(self.Predict(X_train), Y_train)
            
            print("Model Accuracy at epoch No. {} is : {} ".format(e, acc))
        return np.array(Weight_M)


## Function call to Build Model

In [26]:
# Creating object of class model
M = Model()

# model.fit or train the model
Weight_M = M.Train(X_train, Y_train, 15000, 0.0001)
print(Weight_M)

Model Accuracy at epoch No. 0 is : 0.6217008797653959 
Model Accuracy at epoch No. 1 is : 0.6920821114369502 
Model Accuracy at epoch No. 2 is : 0.6832844574780058 
Model Accuracy at epoch No. 3 is : 0.6920821114369502 
Model Accuracy at epoch No. 4 is : 0.6920821114369502 
Model Accuracy at epoch No. 5 is : 0.6920821114369502 
Model Accuracy at epoch No. 6 is : 0.6950146627565983 
Model Accuracy at epoch No. 7 is : 0.6920821114369502 
Model Accuracy at epoch No. 8 is : 0.6950146627565983 
Model Accuracy at epoch No. 9 is : 0.6920821114369502 
Model Accuracy at epoch No. 10 is : 0.6950146627565983 
Model Accuracy at epoch No. 11 is : 0.6920821114369502 
Model Accuracy at epoch No. 12 is : 0.6950146627565983 
Model Accuracy at epoch No. 13 is : 0.6920821114369502 
Model Accuracy at epoch No. 14 is : 0.6950146627565983 
Model Accuracy at epoch No. 15 is : 0.6920821114369502 
Model Accuracy at epoch No. 16 is : 0.6950146627565983 
Model Accuracy at epoch No. 17 is : 0.6920821114369502 
Mo

Model Accuracy at epoch No. 213 is : 0.6950146627565983 
Model Accuracy at epoch No. 214 is : 0.6774193548387096 
Model Accuracy at epoch No. 215 is : 0.6979472140762464 
Model Accuracy at epoch No. 216 is : 0.6979472140762464 
Model Accuracy at epoch No. 217 is : 0.6979472140762464 
Model Accuracy at epoch No. 218 is : 0.6979472140762464 
Model Accuracy at epoch No. 219 is : 0.6950146627565983 
Model Accuracy at epoch No. 220 is : 0.6832844574780058 
Model Accuracy at epoch No. 221 is : 0.6979472140762464 
Model Accuracy at epoch No. 222 is : 0.6979472140762464 
Model Accuracy at epoch No. 223 is : 0.6979472140762464 
Model Accuracy at epoch No. 224 is : 0.6715542521994134 
Model Accuracy at epoch No. 225 is : 0.6979472140762464 
Model Accuracy at epoch No. 226 is : 0.6979472140762464 
Model Accuracy at epoch No. 227 is : 0.6979472140762464 
Model Accuracy at epoch No. 228 is : 0.6979472140762464 
Model Accuracy at epoch No. 229 is : 0.6950146627565983 
Model Accuracy at epoch No. 230

Model Accuracy at epoch No. 421 is : 0.7008797653958945 
Model Accuracy at epoch No. 422 is : 0.6979472140762464 
Model Accuracy at epoch No. 423 is : 0.7008797653958945 
Model Accuracy at epoch No. 424 is : 0.7008797653958945 
Model Accuracy at epoch No. 425 is : 0.7008797653958945 
Model Accuracy at epoch No. 426 is : 0.7008797653958945 
Model Accuracy at epoch No. 427 is : 0.6979472140762464 
Model Accuracy at epoch No. 428 is : 0.7008797653958945 
Model Accuracy at epoch No. 429 is : 0.7008797653958945 
Model Accuracy at epoch No. 430 is : 0.7008797653958945 
Model Accuracy at epoch No. 431 is : 0.6979472140762464 
Model Accuracy at epoch No. 432 is : 0.7008797653958945 
Model Accuracy at epoch No. 433 is : 0.7008797653958945 
Model Accuracy at epoch No. 434 is : 0.6979472140762464 
Model Accuracy at epoch No. 435 is : 0.7008797653958945 
Model Accuracy at epoch No. 436 is : 0.7008797653958945 
Model Accuracy at epoch No. 437 is : 0.7008797653958945 
Model Accuracy at epoch No. 438

Model Accuracy at epoch No. 644 is : 0.7038123167155426 
Model Accuracy at epoch No. 645 is : 0.7008797653958945 
Model Accuracy at epoch No. 646 is : 0.6891495601173021 
Model Accuracy at epoch No. 647 is : 0.6862170087976539 
Model Accuracy at epoch No. 648 is : 0.6862170087976539 
Model Accuracy at epoch No. 649 is : 0.7008797653958945 
Model Accuracy at epoch No. 650 is : 0.7038123167155426 
Model Accuracy at epoch No. 651 is : 0.7038123167155426 
Model Accuracy at epoch No. 652 is : 0.6979472140762464 
Model Accuracy at epoch No. 653 is : 0.7038123167155426 
Model Accuracy at epoch No. 654 is : 0.6979472140762464 
Model Accuracy at epoch No. 655 is : 0.7038123167155426 
Model Accuracy at epoch No. 656 is : 0.6979472140762464 
Model Accuracy at epoch No. 657 is : 0.6979472140762464 
Model Accuracy at epoch No. 658 is : 0.6862170087976539 
Model Accuracy at epoch No. 659 is : 0.7038123167155426 
Model Accuracy at epoch No. 660 is : 0.7008797653958945 
Model Accuracy at epoch No. 661

Model Accuracy at epoch No. 870 is : 0.7008797653958945 
Model Accuracy at epoch No. 871 is : 0.7038123167155426 
Model Accuracy at epoch No. 872 is : 0.7008797653958945 
Model Accuracy at epoch No. 873 is : 0.7067448680351907 
Model Accuracy at epoch No. 874 is : 0.7008797653958945 
Model Accuracy at epoch No. 875 is : 0.7038123167155426 
Model Accuracy at epoch No. 876 is : 0.7008797653958945 
Model Accuracy at epoch No. 877 is : 0.7096774193548387 
Model Accuracy at epoch No. 878 is : 0.7008797653958945 
Model Accuracy at epoch No. 879 is : 0.7067448680351907 
Model Accuracy at epoch No. 880 is : 0.7008797653958945 
Model Accuracy at epoch No. 881 is : 0.7067448680351907 
Model Accuracy at epoch No. 882 is : 0.7008797653958945 
Model Accuracy at epoch No. 883 is : 0.7126099706744868 
Model Accuracy at epoch No. 884 is : 0.7008797653958945 
Model Accuracy at epoch No. 885 is : 0.7038123167155426 
Model Accuracy at epoch No. 886 is : 0.7008797653958945 
Model Accuracy at epoch No. 887

...

Model Accuracy at epoch No. 13955 is : 0.7448680351906158 
Model Accuracy at epoch No. 13956 is : 0.7478005865102639 
Model Accuracy at epoch No. 13957 is : 0.7595307917888563 
Model Accuracy at epoch No. 13958 is : 0.7448680351906158 
Model Accuracy at epoch No. 13959 is : 0.8240469208211144 
Model Accuracy at epoch No. 13960 is : 0.7624633431085044 
Model Accuracy at epoch No. 13961 is : 0.7448680351906158 
Model Accuracy at epoch No. 13962 is : 0.7478005865102639 
Model Accuracy at epoch No. 13963 is : 0.7448680351906158 
Model Accuracy at epoch No. 13964 is : 0.750733137829912 
Model Accuracy at epoch No. 13965 is : 0.7478005865102639 
Model Accuracy at epoch No. 13966 is : 0.750733137829912 
Model Accuracy at epoch No. 13967 is : 0.7595307917888563 
Model Accuracy at epoch No. 13968 is : 0.7448680351906158 
Model Accuracy at epoch No. 13969 is : 0.8240469208211144 
Model Accuracy at epoch No. 13970 is : 0.7624633431085044 
Model Accuracy at epoch No. 13971 is : 0.7448680351906158 

Model Accuracy at epoch No. 14171 is : 0.8240469208211144 
Model Accuracy at epoch No. 14172 is : 0.7478005865102639 
Model Accuracy at epoch No. 14173 is : 0.8269794721407625 
Model Accuracy at epoch No. 14174 is : 0.8240469208211144 
Model Accuracy at epoch No. 14175 is : 0.7888563049853372 
Model Accuracy at epoch No. 14176 is : 0.8240469208211144 
Model Accuracy at epoch No. 14177 is : 0.7478005865102639 
Model Accuracy at epoch No. 14178 is : 0.7595307917888563 
Model Accuracy at epoch No. 14179 is : 0.7595307917888563 
Model Accuracy at epoch No. 14180 is : 0.7595307917888563 
Model Accuracy at epoch No. 14181 is : 0.7478005865102639 
Model Accuracy at epoch No. 14182 is : 0.8240469208211144 
Model Accuracy at epoch No. 14183 is : 0.7536656891495601 
Model Accuracy at epoch No. 14184 is : 0.7595307917888563 
Model Accuracy at epoch No. 14185 is : 0.7565982404692082 
Model Accuracy at epoch No. 14186 is : 0.7478005865102639 
Model Accuracy at epoch No. 14187 is : 0.824046920821114

Model Accuracy at epoch No. 14353 is : 0.7565982404692082 
Model Accuracy at epoch No. 14354 is : 0.7478005865102639 
Model Accuracy at epoch No. 14355 is : 0.7536656891495601 
Model Accuracy at epoch No. 14356 is : 0.7536656891495601 
Model Accuracy at epoch No. 14357 is : 0.7565982404692082 
Model Accuracy at epoch No. 14358 is : 0.7565982404692082 
Model Accuracy at epoch No. 14359 is : 0.7536656891495601 
Model Accuracy at epoch No. 14360 is : 0.7565982404692082 
Model Accuracy at epoch No. 14361 is : 0.7565982404692082 
Model Accuracy at epoch No. 14362 is : 0.7536656891495601 
Model Accuracy at epoch No. 14363 is : 0.7565982404692082 
Model Accuracy at epoch No. 14364 is : 0.7565982404692082 
Model Accuracy at epoch No. 14365 is : 0.7536656891495601 
Model Accuracy at epoch No. 14366 is : 0.7595307917888563 
Model Accuracy at epoch No. 14367 is : 0.7565982404692082 
Model Accuracy at epoch No. 14368 is : 0.7536656891495601 
Model Accuracy at epoch No. 14369 is : 0.759530791788856

Model Accuracy at epoch No. 14586 is : 0.7565982404692082 
Model Accuracy at epoch No. 14587 is : 0.7595307917888563 
Model Accuracy at epoch No. 14588 is : 0.7800586510263929 
Model Accuracy at epoch No. 14589 is : 0.7565982404692082 
Model Accuracy at epoch No. 14590 is : 0.7595307917888563 
Model Accuracy at epoch No. 14591 is : 0.7800586510263929 
Model Accuracy at epoch No. 14592 is : 0.8211143695014663 
Model Accuracy at epoch No. 14593 is : 0.7536656891495601 
Model Accuracy at epoch No. 14594 is : 0.7448680351906158 
Model Accuracy at epoch No. 14595 is : 0.7565982404692082 
Model Accuracy at epoch No. 14596 is : 0.7595307917888563 
Model Accuracy at epoch No. 14597 is : 0.7448680351906158 
Model Accuracy at epoch No. 14598 is : 0.8211143695014663 
Model Accuracy at epoch No. 14599 is : 0.8181818181818182 
Model Accuracy at epoch No. 14600 is : 0.7448680351906158 
Model Accuracy at epoch No. 14601 is : 0.8211143695014663 
Model Accuracy at epoch No. 14602 is : 0.750733137829912

Model Accuracy at epoch No. 14824 is : 0.7448680351906158 
Model Accuracy at epoch No. 14825 is : 0.7536656891495601 
Model Accuracy at epoch No. 14826 is : 0.7478005865102639 
Model Accuracy at epoch No. 14827 is : 0.7800586510263929 
Model Accuracy at epoch No. 14828 is : 0.7800586510263929 
Model Accuracy at epoch No. 14829 is : 0.7595307917888563 
Model Accuracy at epoch No. 14830 is : 0.7448680351906158 
Model Accuracy at epoch No. 14831 is : 0.7536656891495601 
Model Accuracy at epoch No. 14832 is : 0.7478005865102639 
Model Accuracy at epoch No. 14833 is : 0.7478005865102639 
Model Accuracy at epoch No. 14834 is : 0.7478005865102639 
Model Accuracy at epoch No. 14835 is : 0.7536656891495601 
Model Accuracy at epoch No. 14836 is : 0.7478005865102639 
Model Accuracy at epoch No. 14837 is : 0.7478005865102639 
Model Accuracy at epoch No. 14838 is : 0.7478005865102639 
Model Accuracy at epoch No. 14839 is : 0.7478005865102639 
Model Accuracy at epoch No. 14840 is : 0.753665689149560

## Saving model Weights_M

In [27]:
# Saving MOdel weights
print("Shape Of Weights_M : ",Weight_M.shape)
Save_Weights = np.array(Weight_M)

np.savetxt("../saved_weights/Weights_M.csv", Save_Weights, delimiter=",")

Shape Of Weights_M :  (15000, 86)


# Validating Model Over Training Set

In [28]:
# It is Bad practice to predict output over the train Datasets.
# It we divide training dataset into train and test then model will have only few entries to learn
# We will test model on both training and test datasets by considering train dataset as validation dataset

# function call to Predict the labels
Y_pred = M.Predict(X_train)

print("True Values from Train Dataset : ")
print(Y_train)

print("Predicted Values Over Train Features : ")
print(Y_pred)

acc = accuracy_score(M.Predict(X_train), Y_train)
print("Model Accuracy for Training dataset : ", acc)

True Values from Train Dataset : 
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1.
 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1.
 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1.
 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1.
 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1.
 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0.
 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1.
 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1.
 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1.
 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1.
 0. 1. 1. 1. 0. 0

# Evauation of model over the test set

In [29]:
# As we have to submit predictions over test dataset Call Predict function by passing X_test features as parameter
# we can not compute accuracy but fine

In [30]:
# function call to Predict the Labels Over test dataset
Y_pred_sub = M.Predict(X_test)

print("Predicted Labels for Test Features are : ")
print(Y_pred_sub)

Predicted Labels for Test Features are : 
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 1 1 1 1 1 1 1 1]


## Create Final Submission

In [31]:
print("Final Solution for Submission...")

# For each phone Id we have its respective Label Y_pred_sub
submission = pd.DataFrame({'PhoneId':test_new['PhoneId'], 'Class':Y_pred_sub})
submission = submission[['PhoneId', 'Class']]

# Dataframe
submission.head()

Final Solution for Submission...


,PhoneId,Class
0,3,1
1,11,1
2,13,1
3,16,1
4,19,1


In [32]:
# Write Solution to Submission.csv
# This is file to Commit
submission.to_csv("../submission.csv", index=False)

In [33]:
print("End of program...")

End of program...
